In [1]:
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, value

In [2]:
path = "data/parts/"

# Load data using pandas
cpus = pd.read_csv(path + "CPU_Data.csv")
gpus = pd.read_csv(path + "GPU_Data.csv")
rams = pd.read_csv(path + "RAM_Data.csv")

motherboards = pd.read_csv(path + "MOBO_Data.csv")
psus = pd.read_csv(path + "PSU_Data.csv")
cases = pd.read_csv(path + "Case_Data.csv")
storage = pd.read_csv(path + "Storage_Data.csv")

games = pd.read_csv("data/games/top300.csv")

# Return original value if conversion fails
def safe_to_numeric(val):
    try:
        return pd.to_numeric(val)
    except (ValueError, TypeError):
        return val  

# Convert numeric columns from string to numeric types
cpus = cpus.apply(safe_to_numeric)
gpus = gpus.apply(safe_to_numeric)
rams = rams.apply(safe_to_numeric)
cases = cases.apply(safe_to_numeric)
motherboards = motherboards.apply(safe_to_numeric)
psus = psus.apply(safe_to_numeric)
storage = storage.apply(safe_to_numeric)
games = games.apply(safe_to_numeric)

In [4]:
display(cases['Size'].unique())

array(['Micro ATX', 'ATX', 'EATX', 'Mini ITX'], dtype=object)

In [6]:
case_dict = {
    'Micro ATX': 'MATX',
    'Mini ITX': 'ITX'
}

cases['Size'] = cases['Size'].map(case_dict).fillna(cases['Size'])
display(cases['Size'].unique())

array(['MATX', 'ATX', 'EATX', 'ITX'], dtype=object)

In [7]:
display(cases)

,Brand,Model,Size,Color,Price
0,Armaggeddon,Aquaron Duplex One,MATX,"['Black', 'White']",124
1,Armaggeddon,Duplex Pro,MATX,"['Black', 'White','Pink']",114
2,Armaggeddon,Aquaron Wibo,MATX,"['Black', 'White']",124
3,Armaggeddon,Aquaron,MATX,"['Black', 'White','Pink']",114
4,Armaggeddon,Aquaron X Curve,MATX,"['Black', 'White']",174
5,Armaggeddon,Tron Holo 3,MATX,"['Black', 'White']",154
6,Armaggeddon,Tessaraxx Core 12 Air,ATX,"['Black', 'White']",194
7,Armaggeddon,Aquaron Pro,ATX,"['Black', 'White','Pink']",164
8,Armaggeddon,Aqaron Xtreme,EATX,"['Black', 'White','Pink']",294
9,Armaggeddon,Tritron Pro,ATX,['Black'],119


In [8]:
cases.to_csv(f'{path}Case_Data.csv', index=False)

In [3]:
from pulp import *

# Variables
x = [LpVariable(f"x_{i}", cat="Binary") for i in range(3)]
y = LpVariable("y", lowBound=0, upBound=10, cat="Integer")
z = [LpVariable(f"z_{i}", cat="Integer") for i in range(3)]

prob = LpProblem("ActivationExample", LpMaximize)

# Only one item selected
prob += lpSum(x) == 1

# Activation constraints
for i in range(3):
    prob += z[i] <= y
    prob += z[i] <= x[i] * 10
    prob += z[i] >= y - (1 - x[i]) * 10
    prob += z[i] >= 0

# Example objective: maximize sum of z[i]
prob += lpSum(z)

prob.solve()
print([x[i].value() for i in range(3)], y.value(), [z[i].value() for i in range(3)])

[0.0, 0.0, 1.0] 10.0 [-0.0, -0.0, 10.0]
